In [1]:
from google.colab import drive
import os

# 1. 드라이브 연결
drive.mount('/content/drive')
!pip install streamlit pyngrok ultralytics &>/dev/null

Mounted at /content/drive


In [2]:
import shutil
import os

# 1. 경로 설정
# 방금 학습한 따끈따끈한 모델 (Source)
source_path = '/content/drive/MyDrive/cv2/runs/tomato_3class_final/weights/best.pt'

# 웹 앱이 사용 중인 모델 위치 (Destination) - 덮어씌울 곳
destination_path = '/content/drive/MyDrive/cv2/TomatoDoctor_Web/models/ripeness_model.pt'

# 2. 파일 복사 (덮어쓰기)
if os.path.exists(source_path):
    print(f"📦 모델 교체 작업을 시작합니다...")
    print(f"원본: {source_path}")

    shutil.copy(source_path, destination_path)

    print(f"✅ 교체 완료! 대상: {destination_path}")
    print("🚀 이제 웹 앱(Streamlit)을 실행하면 '저해상도 강화 모델'이 작동합니다.")
else:
    print(f"❌ 에러: 원본 모델 파일이 아직 없습니다.\n경로 확인: {source_path}")

📦 모델 교체 작업을 시작합니다...
원본: /content/drive/MyDrive/cv2/runs/tomato_3class_final/weights/best.pt
✅ 교체 완료! 대상: /content/drive/MyDrive/cv2/TomatoDoctor_Web/models/ripeness_model.pt
🚀 이제 웹 앱(Streamlit)을 실행하면 '저해상도 강화 모델'이 작동합니다.


In [5]:
# [Cell] 파일 업로더 디자인(검은박스) 수정 완료된 app.py
%%writefile /content/drive/MyDrive/cv2/TomatoDoctor_Web/app.py

import streamlit as st
from ultralytics import YOLO
from PIL import Image
import pandas as pd
import os

# 1. 페이지 설정
st.set_page_config(page_title="우리 농장 토마토 주치의", layout="wide")

# 2. 스타일 (CSS) - 파일 업로더까지 완벽 커스텀
st.markdown("""
    <style>
    /* [배경] 눈부신 흰색 대신 따뜻한 아이보리색 적용 */
    .stApp {
        background-color: #fffbf0 !important;
    }

    /* [기본 글씨] 무조건 검정 */
    html, body, p, h1, h2, h3, h4, h5, h6, div, span, label, li, small {
        font-family: 'Malgun Gothic', sans-serif;
        color: #2d3436 !important;
    }

    /* 제목 스타일 */
    .main-title {
        font-size: 3.5rem;
        font-weight: 900;
        color: #d63031 !important;
        text-align: center;
        text-shadow: 2px 2px 0px #fff;
    }
    .sub-msg {
        font-size: 1.6rem;
        text-align: center;
        font-weight: 700;
        margin-bottom: 30px;
        color: #636e72 !important;
    }

    /* [선택 버튼] 입체 카드 스타일 */
    div[role="radiogroup"] {
        background-color: transparent;
        padding: 10px;
        display: flex;
        gap: 15px;
    }
    div[role="radiogroup"] label {
        background-color: #ffffff !important;
        padding: 20px 30px !important;
        border-radius: 15px !important;
        border: 3px solid #b2bec3 !important;
        box-shadow: 0px 5px 0px #b2bec3 !important;
        transition: all 0.2s;
        margin-right: 15px !important;
        width: 100%;
    }
    div[role="radiogroup"] label[data-checked="true"] {
        background-color: #ffeaa7 !important; /* 노란색 */
        border: 3px solid #e17055 !important;
        box-shadow: 0px 5px 0px #e17055 !important;
        transform: translateY(2px);
    }
    div[role="radiogroup"] label p {
        font-size: 1.6rem !important;
        font-weight: 900 !important;
        text-align: center;
        margin: 0px !important;
    }
    div[role="radiogroup"] label:hover {
        background-color: #fff7d1 !important;
    }

    /* ------------------------------------------------------- */
    /* [핵심 수정] 파일 업로더(Browse files) 스타일 강제 변경 */
    /* ------------------------------------------------------- */

    /* 1. 드롭존(박스) 배경을 흰색으로, 테두리는 진한 점선 */
    [data-testid='stFileUploaderDropzone'] {
        background-color: #ffffff !important;
        border: 3px dashed #2d3436 !important;
        border-radius: 15px !important;
        padding: 30px !important; /* 내부 여백 넉넉하게 */
    }

    /* 2. 박스 안의 모든 글씨(Drag and drop...) 검정색 & 굵게 */
    [data-testid='stFileUploaderDropzone'] div,
    [data-testid='stFileUploaderDropzone'] span,
    [data-testid='stFileUploaderDropzone'] small {
        color: #000000 !important;
        font-weight: 700 !important;
        font-size: 1.2rem !important;
    }

    /* 3. 'Browse files' 버튼 디자인 */
    [data-testid='stFileUploaderDropzone'] button {
        background-color: #ffeaa7 !important; /* 개나리색 */
        color: #000000 !important; /* 글씨 검정 */
        border: 2px solid #e17055 !important; /* 테두리 주황 */
        border-radius: 8px !important;
        font-weight: 900 !important;
        font-size: 1.1rem !important;
        padding: 10px 20px !important;
    }
    /* ------------------------------------------------------- */

    /* 결과 박스 스타일 */
    .result-box {
        background-color: #e3f9e5;
        padding: 20px;
        border-radius: 15px;
        border: 3px solid #00b894;
        font-size: 1.4rem;
        margin-bottom: 20px;
        box-shadow: 3px 3px 0px rgba(0,0,0,0.1);
    }
    .detail-box {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 15px;
        border: 3px solid #74b9ff;
        font-size: 1.3rem;
        box-shadow: 3px 3px 0px rgba(0,0,0,0.1);
    }
    .warning-box {
        background-color: #fff5e6;
        padding: 25px;
        border-radius: 15px;
        border: 3px solid #e17055;
        font-size: 1.4rem;
        line-height: 1.8;
        box-shadow: 3px 3px 0px rgba(0,0,0,0.1);
    }

    /* 질병 카드 */
    .disease-card {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 10px;
        border: 2px solid #636e72;
        border-left: 15px solid #ffc107;
        margin-bottom: 15px;
    }
    .disease-title { font-size: 1.6rem; font-weight: 900; color: #d35400 !important; }
    .action-step { font-size: 1.3rem; margin-top: 10px; line-height: 1.6; font-weight: 600;}
    </style>
""", unsafe_allow_html=True)

# 3. 데이터베이스
info_db = {
    'b_fully_ripened': {'kor': '🔴 큰 놈 (완숙)', 'type': 'ripe'},
    'b_half_ripened':  {'kor': '🟠 큰 놈 (반숙)', 'type': 'ripe'},
    'b_green':         {'kor': '🟢 큰 놈 (풋토마토)', 'type': 'ripe'},
    'l_fully_ripened': {'kor': '🔴 작은 놈 (완숙)', 'type': 'ripe'},
    'l_half_ripened':  {'kor': '🟠 작은 놈 (반숙)', 'type': 'ripe'},
    'l_green':         {'kor': '🟢 작은 놈 (풋토마토)', 'type': 'ripe'},

    'early_blight': {
        'kor': '겹무늬병 (이른마름병)', 'type': 'disease',
        'cause': '흙 속에 있던 균이 빗물에 튀어서 생깁니다.',
        'action': '1. 땅에 닿은 병든 잎은 따서 버리세요.<br>2. 하우스 환기를 시키세요.<br>3. 만코제브 같은 살균제를 뿌리세요.'
    },
    'late_blight': {
        'kor': '역병 (늦은마름병)', 'type': 'disease',
        'cause': '습하고 서늘할 때 급속도로 번지는 무서운 병입니다.',
        'action': '1. 증상이 보이면 포기 전체를 바로 뽑아 버려야 합니다.<br>2. 물을 끊고 건조하게 관리하세요.<br>3. 역병 전용 약제를 즉시 살포하세요.'
    },
    'leaf_mold': {
        'kor': '잎곰팡이병', 'type': 'disease',
        'cause': '습도가 높을 때 잎 뒷면에 곰팡이가 핍니다.',
        'action': '1. 환기가 제일 중요합니다. 통풍이 잘 되게 곁순을 따주세요.<br>2. 오전에 물을 주고 저녁엔 잎을 말리세요.'
    },
    'yellow_leaf_curl_virus': {
        'kor': '황화잎말림병 (바이러스)', 'type': 'disease',
        'cause': '담배가루이(하얀 날파리)가 옮기는 바이러스입니다.',
        'action': '1. 약이 없습니다. 걸린 토마토는 뽑아서 태우거나 묻으세요.<br>2. 병보다는 담배가루이를 잡는 약을 쳐야 합니다.<br>3. 하우스 주변 잡초를 제거하세요.'
    },
    'powdery_mildew': {
        'kor': '흰가루병', 'type': 'disease',
        'cause': '밀가루를 뿌린 것처럼 하얗게 변합니다.',
        'action': '1. 하얀 게 보이면 잎을 따주세요.<br>2. 난황유(계란노른자+식용유)나 유황 훈증기를 쓰세요.'
    },
    'leaf_miner': {
        'kor': '잎굴파리 피해', 'type': 'disease',
        'cause': '파리 애벌레가 잎 속에 그림을 그리듯 갉아먹습니다.',
        'action': '1. 그림 그려진 잎은 보이는 대로 따서 없애세요.<br>2. 노란색 끈끈이 트랩을 설치하세요.'
    },
    'miner': {'kor': '잎굴파리 피해', 'type': 'disease', 'action': '잎 제거 및 끈끈이 트랩 설치'},
    'blight': {'kor': '마름병류', 'type': 'disease', 'action': '종합 살균제 살포 및 환기 철저'},
    'healthy': {'kor': '정상', 'type': 'healthy', 'action': ''}
}

def get_info(eng_name):
    clean_key = eng_name.lower().strip().replace(' ', '_')
    if clean_key in info_db: return info_db[clean_key]
    for db_key in info_db:
        if db_key in clean_key: return info_db[db_key]
    return {'kor': eng_name, 'type': 'unknown', 'cause': '정보 없음', 'action': '전문가와 상의하세요.'}

# 4. 화면 구성
st.markdown('<div class="main-title">🍅 우리 농장 토마토 주치의</div>', unsafe_allow_html=True)
st.markdown('<div class="sub-msg">사진 속 영어가 어려우시면, 오른쪽 글씨만 보세요.<br>병이 있으면 처방전까지 써드립니다.</div>', unsafe_allow_html=True)

@st.cache_resource
def load_models():
    models = {}
    ripe_path = "models/ripeness_model.pt"
    disease_path = "models/disease_model.pt"
    if os.path.exists(ripe_path): models['ripe'] = YOLO(ripe_path)
    if os.path.exists(disease_path): models['disease'] = YOLO(disease_path)
    return models

with st.spinner("진료 가방 챙기는 중입니다..."):
    models = load_models()

st.sidebar.header("🔧 설정")
conf_val = st.sidebar.slider("민감도 조절", 0.1, 1.0, 0.4, 0.1)

# 5. 입력 방식 선택
st.write("---")
st.markdown('<h2 style="font-weight:900; margin-bottom:15px;">👇 사진을 어떻게 올리시겠습니까?</h2>', unsafe_allow_html=True)

input_method = st.radio(
    "입력 방식을 선택하세요:",
    ("📸 현장에서 바로 찍기", "📂 앨범에서 사진 고르기"),
    label_visibility="collapsed",
    horizontal=True
)

uploaded_file = None

if input_method == "📸 현장에서 바로 찍기":
    img_file = st.camera_input("카메라 버튼을 눌러주세요")
    if img_file is not None:
        uploaded_file = img_file
else:
    # 안내 문구를 좀 더 크게
    st.markdown("<h4 style='font-weight:bold; margin-bottom:10px;'>📂 아래 상자를 눌러서 사진을 고르세요</h4>", unsafe_allow_html=True)
    img_file = st.file_uploader("여기를 눌러 앨범에서 사진을 선택하세요", type=["jpg", "png", "jpeg"], label_visibility="collapsed")
    if img_file is not None:
        uploaded_file = img_file

# 6. 추론 로직
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    col_img, col_txt = st.columns([1, 1])

    with st.spinner("꼼꼼하게 진찰 중입니다..."):
        res_ripe = None; plot_ripe = None
        res_disease = None; plot_disease = None

        if 'ripe' in models:
            res_ripe = models['ripe'].predict(image, conf=conf_val, iou=0.5, agnostic_nms=True)[0]
            plot_ripe = res_ripe.plot()[:, :, ::-1]
        if 'disease' in models:
            res_disease = models['disease'].predict(image, conf=conf_val, iou=0.5, agnostic_nms=True)[0]
            plot_disease = res_disease.plot()[:, :, ::-1]

    # 왼쪽: 사진
    with col_img:
        st.subheader("📷 사진 확인")
        tab1, tab2 = st.tabs(["토마토 상태", "병해충"])
        with tab1:
            if plot_ripe is not None: st.image(plot_ripe, caption="네모 박스 위 영어는 무시하셔도 됩니다.", use_container_width=True)
        with tab2:
            if plot_disease is not None: st.image(plot_disease, caption="아픈 부위 표시", use_container_width=True)

    # 오른쪽: 결과
    with col_txt:
        st.subheader("📝 상세 진단표")

        # [1] 숙성도
        if res_ripe and len(res_ripe.boxes) > 0:
            cls_r = res_ripe.boxes.cls.cpu().numpy().astype(int)
            names_r = res_ripe.names

            count_dict = {}
            total_count = len(cls_r)
            fully_count = 0

            for c in cls_r:
                eng = names_r[c]
                kor_name = get_info(eng)['kor']
                count_dict[kor_name] = count_dict.get(kor_name, 0) + 1
                if 'fully' in eng.lower() or 'ripened' in eng.lower(): fully_count += 1

            ratio = (fully_count / total_count) * 100

            result_html = ""
            if ratio >= 70:
                result_html = f"<h3>🎉 <b>수확 하세요! ({int(ratio)}% 완숙)</b></h3>지금 따야 제일 맛있습니다."
            elif ratio >= 30:
                result_html = f"<h3>⏳ <b>좀 더 기다리세요.</b></h3>익어가는 중입니다."
            else:
                result_html = f"<h3>🌱 <b>아직 파릇파릇합니다.</b></h3>더 키우셔야겠네요."

            st.markdown(f'<div class="result-box">{result_html}</div>', unsafe_allow_html=True)

            detail_list = ""
            for k, v in count_dict.items():
                detail_list += f"<li>{k}: <b>{v}개</b></li>"

            detail_html = f"""
            <div class="detail-box">
                <b>🔎 개수 확인</b>
                <ul style="margin-top:5px; margin-bottom:0px; padding-left:20px;">
                    {detail_list}
                </ul>
                <div style="margin-top:10px; border-top:2px dashed #b2bec3; padding-top:5px;">
                   👉 총 <b>{total_count}개</b>
                </div>
            </div>
            """
            st.markdown(detail_html, unsafe_allow_html=True)

        else:
            if res_ripe is not None: st.warning("토마토 열매가 안 보입니다.")

        st.markdown("---")

        # [2] 질병
        if res_disease and len(res_disease.boxes) > 0:
            cls_d = res_disease.boxes.cls.cpu().numpy().astype(int)
            names_d = res_disease.names

            detected_infos = []
            is_healthy = True

            for c in cls_d:
                eng = names_d[c]
                info = get_info(eng)
                if info['type'] != 'healthy':
                    is_healthy = False
                    if info not in detected_infos: detected_infos.append(info)

            if not is_healthy:
                warning_html = """
                <h3>🚨 <b>아이고, 병이 발견되었습니다!</b></h3>
                아래 처방전을 꼭 읽어보세요.
                """
                st.markdown(f'<div class="warning-box">{warning_html}</div>', unsafe_allow_html=True)

                for info in detected_infos:
                    st.markdown(f"""
                    <div class="disease-card">
                        <div class="disease-title">💊 {info['kor']}</div>
                        <p><b>[원인]</b> {info['cause']}</p>
                        <hr style="border-top: 2px dashed #d35400;">
                        <div class="action-step"><b>[농부님 대처법]</b><br>{info['action']}</div>
                    </div>
                    """, unsafe_allow_html=True)
            else:
                st.success("✨ 아주 건강합니다! 병이 하나도 없네요.")
        else:
            if res_disease is not None: st.info("병해충은 발견되지 않았습니다. 안심하세요.")

Overwriting /content/drive/MyDrive/cv2/TomatoDoctor_Web/app.py


In [6]:
from google.colab import drive
from pyngrok import ngrok
import os

# 1. 드라이브 연결
drive.mount('/content/drive')

# 2. 작업 폴더로 이동 (드라이브에 저장된 폴더)
%cd /content/drive/MyDrive/cv2/TomatoDoctor_Web

# 3. 앱 실행
ngrok.set_auth_token("38VERcuvWjWQgYY6FhkqrjvUIZ8_4hmYLpHzxT1jE7fWdzPVb") # 토큰 필요 시 주석 해제
ngrok.kill()
public_url = ngrok.connect(8501).public_url
print(f"🚀 바로가기: {public_url}")
!streamlit run app.py &>/dev/null&

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cv2/TomatoDoctor_Web
🚀 바로가기: https://proliferous-stasia-subdiaphragmatically.ngrok-free.dev


In [7]:
# [Cell] Streamlit & Ngrok 강제 재시작 키트
import os
from pyngrok import ngrok
import time

# 1. 기존 프로세스 전부 살해 (좀비 프로세스 정리)
print("💀 기존 서버를 정리하는 중...")
os.system("pkill -f streamlit")
ngrok.kill()
time.sleep(2) # 확실히 죽을 때까지 대기

# 2. Streamlit 백그라운드 실행
print("🚀 Streamlit 앱을 다시 켜는 중...")
# 로그를 남겨서 에러 확인 가능하게 함
get_ipython().system_raw("streamlit run /content/drive/MyDrive/cv2/TomatoDoctor_Web/app.py > streamlit_logs.txt 2>&1 &")

# 3. 서버가 켜질 때까지 잠시 대기 (중요!)
print("⏳ 서버 예열 중 (5초 대기)...")
time.sleep(5)

# 4. Ngrok 터널 연결
try:
    # 8501 포트 열기
    public_url = ngrok.connect(8501).public_url
    print(f"\n🎉 복구 완료! 아래 주소로 접속하세요:\n👉 {public_url}")
except Exception as e:
    print(f"⚠️ 터널 연결 실패: {e}")
    print("Ngrok 토큰이 만료되었을 수 있습니다. 토큰 설정을 다시 확인해주세요.")

# 5. 실행 로그 확인 (혹시 또 안 되면 이걸 봐야 함)
print("\n[시스템 로그 확인]")
os.system("head -n 20 streamlit_logs.txt")

💀 기존 서버를 정리하는 중...
🚀 Streamlit 앱을 다시 켜는 중...
⏳ 서버 예열 중 (5초 대기)...

🎉 복구 완료! 아래 주소로 접속하세요:
👉 https://proliferous-stasia-subdiaphragmatically.ngrok-free.dev

[시스템 로그 확인]


0